In [1]:
import pandas as pd
import plotly.express as px


df = pd.read_csv("reviews.csv")


# sørg for riktige typer
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df["helpful"] = pd.to_numeric(df["helpful"], errors="coerce").fillna(0).astype(int)

# dato: du har både mm/dd og dd/mm i eksempelet, så vi prøver begge
d1 = pd.to_datetime(df["review_date"], errors="coerce", dayfirst=False)
d2 = pd.to_datetime(df["review_date"], errors="coerce", dayfirst=True)
df["review_date"] = d1.fillna(d2)

df.head()


/var/folders/g4/yk6n2qr95jz52s70_0mf4ydw0000gn/T/ipykernel_47330/3564186074.py:14: UserWarning: Parsing dates in %m/%d/%Y %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  d2 = pd.to_datetime(df["review_date"], errors="coerce", dayfirst=True)


,review_id,rating,review_text,review_date,helpful
0,981e465b-d3ba-4632-9c60-25051efac38a,5,It's good,2025-11-22 01:19:00,0
1,964d3555-9429-4c20-8127-ce3c71ce9273,5,WhatsApp not working well always shows offline...,2025-11-24 20:03:00,0
2,6c28859f-1554-4ca1-9aa8-9d66f204be0a,5,"Oppo not corresponding, share with me the offi...",2025-11-25 06:26:00,0
3,a7efafc3-5871-4020-a398-9cc12cb4072a,5,"Excellent app, great communication super conne...",2025-11-25 18:09:00,0
4,de142b31-a5ad-446f-b7c8-51b264728478,4,simply the ɓest for chat and calls.i love it,2025-11-24 01:10:00,1


In [2]:
fig = px.histogram(df, x="rating", nbins=5, title="Fordeling av rating")
fig.show()